╔══<i><b>Alai-DeepLearning</b></i>════════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 13. RNN-Basis**
# Section 3. RNN에서의 Feed Forward


### _Objective_

1. 순환신경망(RNN)에서 어떤 순서로 동작하는 지 알아보도록 하겠습니다. <br>
2. RNN은 Tensorflow Low-API로 작성하는 것보다, Keras High-API를 위주로 작성하겠습니다.

╚═════════════════════════════════════════╝

In [1]:
import numpy as np
import tensorflow as tf
import keras

from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

Using TensorFlow backend.


<br><br>

# \[ 1. RNN의 Feed Forward  \]
---
---

> *RNN은 시간적 순서(time step)으로 처리되는 연산집합 `Cell`을 순환하며 처리합니다.* <br>
> *Hello라는 Sequence를 출력하는 RNN 모델을 학습해보도록 하겠습니다.*<br>

<br>

## 1. 철자를 Vector로 표현하기
---

* 우리는 철자를 숫자로 표현하기 위해, 가장 간단한 방법 중 하나인 one-hot Vector를 이용하도록 하겠습니다.

### (1) One hot Vector로 표현하기

| Char | index |One-Hot Vector|
|--- |---|---| 
| h | 0 | [1,0,0,0,0] |
| e | 1 | [0,1,0,0,0] |
| l | 2 | [0,0,1,0,0] |
| o | 3 | [0,0,0,1,0] |
| <EOS\> | 4 | [0,0,0,0,1] |

In [0]:
char2vec = {'h':np.array([1,0,0,0,0]),
            "e":np.array([0,1,0,0,0]),
            "l":np.array([0,0,1,0,0]),
            'o':np.array([0,0,0,1,0]),
            '<EOS>':np.array([0,0,0,0,1])}

idx2char = ['h','e','l','o','<EOS>']

In [3]:
print("Character -> Vector : 'l' -> ", char2vec['l'])
print("index -> Character : '2' -> ", idx2char[2])

Character -> Vector : 'l' ->  [0 0 1 0 0]
index -> Character : '2' ->  l


### (2) Embedding Vector로 표현하기

훨씬 많은 단어 혹은 철자를 처리할 때에는 One-hot Vector보다는 `embedding` Layer를 많이 이용합니다.<br> 

In [0]:
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Embedding, Input
from tensorflow.keras.models import Model

In [6]:
K.clear_session()
tf.set_random_seed(1)

inputs = Input(shape=())
embeded = Embedding(input_dim=5,output_dim=2)(inputs)

embeded

model = Model(inputs,embeded)

<class 'tensorflow.python.framework.ops.Tensor'>


`embedding` Layer는 난수로 채워진 Matrix을 만들어 줍니다. 우리는 `embedding` layer에서 받은 Input은 Matrix의 각 행에 대한 인덱스로 행의 값들을 반환하게 됩니다.

In [0]:
# Embedding Layer 내 Weight 가져오기
sess = K.get_session()
graph = sess.graph
embedding_weight = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)[0]

sess.run(embedding_weight)

array([[ 0.00211157,  0.04429672],
       [-0.01521801, -0.02037966],
       [-0.02079648, -0.02168524],
       [-0.0420602 ,  0.04103562],
       [-0.02007725,  0.02599252]], dtype=float32)

|char|index|1번째 임베딩 벡터 값|2번째 임베딩 벡터 값|
|----|----|----|----|
|"h"| 0| 0.002 | 0.044 |
|"e"| 1| -0.015 | -0.020 |
|"l"| 2| -0.021 | -0.022 |
|"o"| 3| -0.042 | 0.041 |
|"<eos\>"| 4| -0.020 | 0.026 |

In [0]:
print("index 0 : {}".format(model.predict([[0]])))
print("index 1 : {}".format(model.predict([[1]])))
print("index 2 : {}".format(model.predict([[2]])))

index 0 : [[0.00211157 0.04429672]]
index 1 : [[-0.01521801 -0.02037966]]
index 2 : [[-0.02079648 -0.02168524]]


예를 들어 한국어로 표현할 수 있는 음절 수는 총 11,172자로 이를 바로 one-hot vector로 표시하면 11,172차원의 공간으로 Mapping되기 때문에, 지나치게 Sparse하게 표현됩니다. <br>
이런 경우 Embedding Layer은 훨씬 작은 차원으로 맵핑시킬 수 있습니다. 위의 Embedding Layer의 Mapping은 Weight로 구성되어 있기 때문에, 처음에는 무작위로 배치되어 있다가, 학습해 감에 따라,각각의 철자의 Embedding Vector가 바뀝니다.


reference : 
1. [How to use word embedding layers for deep learning with keras](https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/)

<br>

## 2. Time Step 별 계산

----

* 보다 직관적인 계산을 위해 철자는 one-hot vector로 구성하여 계산하도록 하겠습니다.<br>
* RNN은 Time Step 별로 가중치를 공유합니다.

### (1) 가중치 구성하기

기학습한 가중치를 이용하도록 하겠습니다. 

In [0]:
### RNN Cell 내 가중치
w_xh = np.array([[-2.6, -1.6, -2.1],
                 [ 1.2,  0.4,  0.3],
                 [ 2.1,  1.9, -0.7],
                 [-1.4, -1.5,  2.5],
                 [-0.9,  0.4, -0.9]])

w_hh = np.array([[-0.5, -2.3,  2.9],
                 [ 1.9,  1.5,  1.7],
                 [-0.7, -1.2,  1.5]])

b_h = np.array([-0.5, -0.4, -1. ])

# Output Layer 내 가중치
w_hy = np.array([[ 0.3, -2.6,  1.2,  2.6, -1.1],
                 [-1.1, -2.4,  2.2,  1.6, -2.4],
                 [-0.4, -3.1, -3. ,  3.6,  3. ]])

b_y = np.array([-1.8, -0.5,  1.3,  0.1,  0.8])

### (2) Time Step 1에서의 계산

In [0]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=-1)

In [0]:
init_h = np.array([0,0,0])
x_0 = char2vec['h']

a_1 = np.dot(init_h, w_hh) + np.dot(x_0,w_xh) + b_h
h_1 = np.tanh(a_1)

y_1 = np.dot(h_1, w_hy) + b_y
o_1 = softmax(y_1)

print("1번째 Timestamp의 hidden : {}".format(h_1))
print("1번째 Timestamp의 output : {}".format(y_1))
print("1번째 Timestamp의 result : {}".format(idx2char[np.argmax(o_1)]))

1번째 Timestamp의 hidden : [-0.99594936 -0.96402758 -0.99594936]
1번째 Timestamp의 output : [-0.63997473  7.49057754  0.97184817 -7.61733016  1.22136241]
1번째 Timestamp의 result : e


### (2) Time Step 2에서의 계산

In [0]:
x_1 = char2vec['e']

a_2 = np.dot(h_1, w_hh) + np.dot(x_1, w_xh) + b_h
h_2 = np.tanh(a_2)

y_2 = np.dot(h_2, w_hy) + b_y
o_2 = softmax(y_2)


print("2번째 Timestamp의 hidden : {}".format(h_2))
print("2번째 Timestamp의 output : {}".format(y_2))
print("2번째 Timestamp의 result : {}".format(idx2char[np.argmax(o_2)]))

2번째 Timestamp의 hidden : [ 0.06340167  0.96673299 -0.99999709]
2번째 Timestamp의 output : [-2.44438695  0.11498748  6.50288586 -1.78837242 -4.58989229]
2번째 Timestamp의 result : l


### (3) Time Step 3에서의 계산

In [0]:
x_2 = char2vec['l']

a_3 = np.dot(h_2, w_hh) + np.dot(x_2,w_xh) + b_h
h_3 = np.tanh(a_3)

y_3 = np.dot(h_3,w_hy)+b_y
o_3 = softmax(y_3)

print("3번째 Timestamp의 hidden : {}".format(h_3))
print("3번째 Timestamp의 output : {}".format(y_3))
print("3번째 Timestamp의 result : {}".format(idx2char[np.argmax(o_3)]))

3번째 Timestamp의 hidden : [ 0.9994564  0.999335  -0.8793026]
3번째 Timestamp의 output : [-2.24771054 -2.7711526   7.33579249  1.1320333  -5.33571385]
3번째 Timestamp의 result : l


### (4) Time Step 4에서의 계산

In [0]:
x_3 = char2vec['l']

a_4 = np.dot(h_3, w_hh) + np.dot(x_3,w_xh) + b_h
h_4 = np.tanh(a_4)

y_4 = np.dot(h_4,w_hy)+b_y
o_4 = softmax(y_4)

print("4번째 Timestamp의 hidden : {}".format(h_4))
print("4번째 Timestamp의 output : {}".format(y_4))
print("4번째 Timestamp의 result : {}".format(idx2char[np.argmax(o_4)]))

4번째 Timestamp의 hidden : [0.99855062 0.9419888  0.91834213]
4번째 Timestamp의 output : [-2.90395935 -8.20386532  1.81560973  7.50944534  0.19584758]
4번째 Timestamp의 result : o


<br><br>

# \[ 2. Keras에서 RNN 구성하기  \]
---
---

> *텐서플로우에서의 RNN 구현은 매우 어렵습니다. 시계열 데이터는 동적인 데이터인 것에 반해, Tensorflow의 Graph는 정적이기 때문입니다.<br>
현재 텐서플로우 내의 RNN 구현체는 Keras 스타일로 대체되고 있습니다. <br>
이에 따라 텐서플로우 내 Keras 메소드들을 이용하여 구현하도록 하겠습니다.*


<br>

## 1. Keras의 순환 신경망 구조 : RNN와 RNN Cell
---

* Keras에서는 연산을 처리하는 Class인 `RNN Cell`과 그것을 Time Step 별로 반복시켜 주는 Wrapper Class `RNN`으로 나누어 구성되어 있습니다.<br>
* 우리는 Cell에서의 인터페이스와 연산을 구성하면, `RNN`을 통해 간단히 Time Step 별로 반복시켜 줄 수 있습니다.

### (1) RNNCell 구성하기

우리는 반복하는 구간인 RNNCell을 아래와 같이 구성할 수 있습니다. <br>
Keras는 하나의 인터페이스로, 꼭 구현해야 하는 요소들이 존재합니다. RNNCell을 구현하기 위해서는 아래 요소들을 지켜주어야 합니다. 이를 지켜주지 않으면 Wrapper Class인 RNN에서 정상적으로 동작하지 않습니다.

In [0]:
from tensorflow.keras.layers import Layer

class RNNCell(Layer):
    """
    Keras에서 Cell을 구성하는 Layer
    
    1. __init__ : Cell에 필요한 hyper-parameter를 구성
       상태 벡터의 크기를 의미하는 self.state_size는 필수적으로 정해주어야 함
       마지막에 super([Layer],self).__init__(**kwargs)를 호출해야함

    2. build : Cell에서 관리하는 Weight들을 선언. 
       마지막에 super([Layer],self).build()를 호출해야함
    
    3. call : Cell의 연산 순서를 정의
    
    """
    def __init__(self, n_units, **kwargs):
        self.n_units = n_units
        self.state_size = n_units
        super(RNNCell, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.w_xh = self.add_weight(name='weight_xh',
                                    shape=(input_shape[-1], self.n_units),
                                    initializer=tf.initializers.glorot_normal())
        self.w_hh = self.add_weight(name='weight_hh',
                                    shape=(self.n_units, self.n_units),
                                    initializer=tf.initializers.orthogonal())
        self.b_h = self.add_weight(name='bias_h',
                                   shape=(self.n_units,),
                                   initializer=tf.initializers.zeros())
        super(RNNCell, self).build(input_shape)
        
    def call(self, inputs, states):
        prev_states = states[0]
        h = (tf.matmul(inputs, self.w_xh) 
             + tf.matmul(prev_states, self.w_hh)
             + self.b_h)
        a = tf.tanh(h)
        return a, [a]

### (2) Keras로 RNN 모델 구성하기

In [0]:
from tensorflow.keras.layers import Input, Dense, RNN

n_inputs = 5 # Input 차원 수
n_steps = 5 # time step의 크기
n_neurons = 3 # Hidden 차원 수 
n_outputs = n_inputs # Output 차원 수 

inputs = Input(shape=(n_steps,n_inputs))
hidden = RNN(RNNCell(n_neurons), return_sequences=True)(inputs)
output = Dense(n_outputs, activation='softmax')(hidden)

`RNN` Wrapper Class에서 반환할 수 있는 형태는 크게 2가지입니다.<br>

* return_sequences : 각 time Step 별 Hidden State의 값을 반환
* return_state : 마지막 time Step의 값을 반환

In [0]:
inputs = Input(shape=(n_steps,n_inputs))
hidden = RNN(RNNCell(n_neurons), return_sequences=False)(inputs)
print("return_Sequences=False 일때의 출력 값 형태: ",hidden.shape)

hidden = RNN(RNNCell(n_neurons), return_sequences=True)(inputs)
print("return_Sequences=True 일때의 출력 값 형태: ",hidden.shape)

hidden = RNN(RNNCell(n_neurons), return_sequences=True, 
             return_state=True)(inputs)
print("return_Sequences=True&return_state=True 일때의 첫번째 출력 값 형태: ",hidden[0].shape)
print("return_Sequences=True&return_state=True 일때의 두번째 출력 값 형태: ",hidden[1].shape)

return_Sequences=False 일때의 출력 값 형태:  (?, 3)
return_Sequences=True 일때의 출력 값 형태:  (?, 5, 3)
return_Sequences=True&return_state=True 일때의 첫번째 출력 값 형태:  (?, 5, 3)
return_Sequences=True&return_state=True 일때의 두번째 출력 값 형태:  (?, 3)


### (3) Keras를 통해 출력값 계산하기

In [0]:
from tensorflow.keras.models import Model

n_inputs = 5 # Input 차원 수
n_steps = 5 # time step의 크기
n_neurons = 3 # Hidden 차원 수 
n_outputs = n_inputs # Output 차원 수 

inputs = Input(shape=(n_steps,n_inputs))
hidden = RNN(RNNCell(n_neurons), return_sequences=True)(inputs)
output = Dense(n_outputs, activation='softmax')(hidden)

model = Model(inputs, output)

# pretrained weight들로 setting
model.set_weights([w_xh, w_hh,b_h,w_hy,b_y]) 

In [0]:
input_values = "hello"
print("입력값 : ",list(input_values))
input_vecs = np.stack([char2vec[char] 
                       for char in input_values])[np.newaxis]

results = model.predict(input_vecs)
result_indices = np.argmax(results,axis=-1)[0]
print("출력값 : ",[idx2char[idx] for idx in result_indices])

입력값 :  ['h', 'e', 'l', 'l', 'o']
출력값 :  ['e', 'l', 'l', 'o', '<EOS>']


### (4) 기존의 Keras API를 활용하기

위와 같이 우리가 직접구현한 RNNCell은 `SimpleRNNCell`로 이미 구현되어 있습니다.<br>

In [0]:
from tensorflow.keras.layers import SimpleRNNCell

inputs = Input(shape=(n_steps,n_inputs))
hidden = RNN(SimpleRNNCell(n_neurons), return_sequences=True)(inputs)
output = Dense(n_outputs, activation='softmax')(hidden)

model = Model(inputs, output)

# pretrained weight들로 setting
model.set_weights([w_xh, w_hh,b_h,w_hy,b_y]) 

In [0]:
input_values = "hello"
print("입력값 : ",list(input_values))
input_vecs = np.stack([char2vec[char] 
                       for char in input_values])[np.newaxis]

results = model.predict(input_vecs)
result_indices = np.argmax(results,axis=-1)[0]
print("출력값 : ",[idx2char[idx] for idx in result_indices])

입력값 :  ['h', 'e', 'l', 'l', 'o']
출력값 :  ['e', 'l', 'l', 'o', '<EOS>']


그리고 Cell에 RNN까지 Wrapping해놓은 것은 `SimpleRNN`이라는 이름으로 제공되고 있습니다.<br>
`SimpleRNN`을 이용하면 보다 간결하게 작성할 수 있습니다.

In [0]:
from tensorflow.keras.layers import SimpleRNN

inputs = Input(shape=(n_steps,n_inputs))
hidden = SimpleRNN(n_neurons, return_sequences=True)(inputs)
output = Dense(n_outputs, activation='softmax')(hidden)

model = Model(inputs, output)

# pretrained weight들로 setting
model.set_weights([w_xh, w_hh,b_h,w_hy,b_y]) 

In [0]:
input_values = "hello"
print("입력값 : ",list(input_values))
input_vecs = np.stack([char2vec[char] 
                       for char in input_values])[np.newaxis]

results = model.predict(input_vecs)
result_indices = np.argmax(results,axis=-1)[0]
print("출력값 : ",[idx2char[idx] for idx in result_indices])

입력값 :  ['h', 'e', 'l', 'l', 'o']
출력값 :  ['e', 'l', 'l', 'o', '<EOS>']
